In [ ]:
import gymnasium as gym
import aero_gym
from aero_gym.tools import evaluate, plotfile, animaterender, animaterender_contour
import matplotlib
import math
import numpy as np
from julia import Main
import logging

In [ ]:
logging.getLogger().setLevel(logging.INFO)

In [ ]:
import sys
sys.path.append('../aero_gym_SB3/')
import trajectory_generators

In [ ]:
matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['image.interpolation'] = 'none'

In [ ]:
t_max = 10
delta_t = 0.1
t = np.linspace(0, t_max, int(t_max/delta_t)+1)

In [ ]:
mylevels = np.concatenate([[-100], np.linspace(-20, 20, num=30), [100]])

## No h_ddot

In [ ]:
env = gym.make(
    'aero_gym/viscous_flow-v0',
    render_mode="grid",
    t_max=t_max,
    delta_t=delta_t,
    Re=200,
    grid_Re=4,
    observe_vorticity_field=True,
    observe_previous_lift=True,
    observe_previous_pressure=True,
    pressure_sensor_positions=[-0.3,-0.2,-0.1,0.0,0.1,0.2,0.3],
    h_ddot_scale=1,
    alpha_ddot_scale=1,
    vorticity_scale=10.0
)

In [ ]:
obs, info = env.reset();
info

In [ ]:
obs, _, _, _, info = env.step([0]);
info

In [ ]:
obs, info, render_list = evaluate(env)
print(f"{info['episode']['t'][0]} seconds")

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)

In [ ]:
xg, yg = Main.eval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels);
anim

## Impulse

In [ ]:
env.reset(options={"h_ddot_prescribed":None,"h_ddot_generator":trajectory_generators.impulse(1.0)});

In [ ]:
obs, info, render_list = evaluate(env)

In [ ]:
env.unwrapped.h_ddot_list

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)
axarr[0,0].plot(info["solver_t_hist"], info["solver_fy_hist"])

In [ ]:
xg, yg = Main.eval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels);
anim

## Step

In [ ]:
env.reset(options={"h_ddot_prescribed":None,"h_ddot_generator":trajectory_generators.constant(0.1)});

In [ ]:
obs, info, render_list = evaluate(env)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)
axarr[0,0].plot(info["solver_t_hist"], info["solver_fy_hist"])

In [ ]:
xg, yg = Main.eval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels);
anim

## Gaussian

In [ ]:
def gaussian(x, a, b, c):
    return a * math.exp(-(x - b) ** 2 / (2 * c ** 2))

def dgaussian(x, a, b, c):
    return a * -(x - b) / c ** 2 * math.exp(-(x - b) ** 2 / (2 * c ** 2))

def ddgaussian(x, a, b, c):
    return a * (x ** 2 + b ** 2 - 2 * b * x - c ** 2) / c ** 4 * math.exp(-(x - b) ** 2 / (2 * c ** 2))

In [ ]:
a = -2/3
b = 3.0
c = 0.4
h_ddot_dgaussian = [0.25 * dgaussian(ti, a, b, c) for ti in t]

a = 1/6
b = 8.0
c = 0.4
alpha_ddot_dgaussian = [ddgaussian(ti, a, b, c) for ti in t]

In [ ]:
env.reset(options={"h_ddot_prescribed":h_ddot_dgaussian});

In [ ]:
obs, info, render_list = evaluate(env, alpha_ddot_prescribed=alpha_ddot_dgaussian)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)
axarr[0,0].plot(info["solver_t_hist"], info["solver_fy_hist"])

In [ ]:
xg, yg = Main.eval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
matplotlib.rcParams['image.interpolation'] = 'none'
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels, animate_every=2);
anim

## Random steps/ramps

In [ ]:
env_random_steps_ramps = gym.make(
    'aero_gym/viscous_flow-v0',
    render_mode="grayscale_array",
    t_max=3,
    delta_t=delta_t,
    observe_vorticity_field=False,
    observe_previous_lift=True,
    observe_previous_pressure=True,
    pressure_sensor_positions=[-0.3,-0.2,-0.1,0.0,0.1,0.2,0.3],
    h_ddot_scale=1,
    alpha_ddot_scale=0.1,
    vorticity_scale=10.0,
    lift_scale=1.0,
    h_ddot_generator=trajectory_generators.random_d_steps_ramps(max_int_amplitude=1.0, max_d_amplitude=1.0)
)

In [ ]:
obs, info = env_random_steps_ramps.reset()

In [ ]:
obs, info, render_list = evaluate(env_random_steps_ramps)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)
axarr[0,0].plot(info["solver_t_hist"], info["solver_fy_hist"])

In [ ]:
xg, yg = Main.eval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels, animate_every=2);
anim

## Point forcing

In [ ]:
env_forcing = gym.make(
    'aero_gym/viscous_flow-v0',
    render_mode="grid",
    t_max=3.0,
    delta_t=0.03,
    observe_vorticity_field=True,
    observe_previous_lift=True,
    observe_previous_lift_error=True,
    alpha_ddot_scale=15.0,
    lift_scale=0.1,
    lift_upper_limit=1.2,
    lift_lower_limit=-1.2,
    lift_termination=True,
    vorticity_scale=10,
    xlim=[-1.25,1.75],
    ylim=[-0.5,0.5],
    initialization_time=5.0,
    sys_reinit_commands='../aero_gym_SB3/julia_sys_reinit_commands_files/julia_system_with_one_prescribed_force_pulse_close.jl'
)

In [ ]:
obs, info, render_list = evaluate(env_forcing)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)
axarr[0,0].plot(info["solver_t_hist"], info["solver_fy_hist"])

In [ ]:
xg, yg = Main.eval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels);
anim

## Reference lift

In [ ]:
t_max_ref = 20
delta_t = 0.1
t_ref = np.linspace(0, t_max_ref, int(t_max_ref/delta_t)+1)

In [ ]:
alpha_ddot_scale = 10.0
alpha = 25 * np.pi / 180
T = alpha / (alpha_ddot_scale * delta_t)
alpha_ddot_for_constant_nonzero_alpha = np.zeros(len(t_ref))
alpha_ddot_for_constant_nonzero_alpha[0] = 1
alpha_ddot_for_constant_nonzero_alpha[int(T / delta_t)] = -1

In [ ]:
env_ref = gym.make(
    'aero_gym/viscous_flow-v0',
    render_mode="grid",
    t_max=t_max_ref,
    delta_t=delta_t,
    observe_vorticity_field=True,
    observe_previous_lift=True,
    observe_previous_lift_error=True,
    h_ddot_scale=0.1,
    alpha_ddot_scale=alpha_ddot_scale,
    reference_lift_generator=trajectory_generators.constant(0.5),
    lift_scale=1,
    lift_upper_limit=2.0,
    lift_lower_limit=-2.0,
    lift_termination=True,
    vorticity_scale=10,
    ylim=[-0.55,0.85]
)

In [ ]:
obs, info, render_list = evaluate(env_ref, alpha_ddot_prescribed = alpha_ddot_for_constant_nonzero_alpha)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)

In [ ]:
xg, yg = Main.eval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels);
anim

## Vortex shedding

In [ ]:
env_vks = gym.make(
    'aero_gym/viscous_flow-v0',
    render_mode="grid",
    t_max=10,
    delta_t=delta_t,
    observe_vorticity_field=True,
    observe_previous_lift=True,
    observe_previous_lift_error=True,
    h_ddot_scale=0.1,
    alpha_ddot_scale=10,
    reference_lift_generator=trajectory_generators.constant(0.7),
    lift_scale=1,
    lift_upper_limit=1.0,
    lift_lower_limit=-0.1,
    lift_termination=True,
    vorticity_scale=10,
    ylim=[-0.55,0.85],
    alpha_init=0.52356,
    initialization_time=10
)

In [ ]:
obs, info, render_list = evaluate(env_vks)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)

In [ ]:
xg, yg = Main.eval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels, alpha_init=0.52356);
anim